wikiAPIを使う代わりにhtmlからすべてのデータを取得する方法に切り替えたverです

In [2]:
# データベース作成のために、まずはwikipediaの「カテゴリー：～年の漫画」のページからタイトルとURLを取得する
# タイトルとurlのリストはpickleでディレクトリ直下に""./manga_title_list.pickle"の形で保存されています
# !pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup

all_manga_list =[]
for year in range(1900,2026):
    manga_list = []
    try:
        url = 'https://ja.wikipedia.org/wiki/Category:' + str(year)+'%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB'
        print(f"The URL for '次のページ' is: {url}")
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        # 下のtry: exceptは、wikipediaのページによってなぜか使用がことなることに対応するための策（網羅できてるかはわかりません）
        try:
            category_div = soup.find('div', class_='mw-category mw-category-columns')
            manga_links = category_div.find_all('a', href=True)
        except:
            try:
                category_div = soup.find('div', class_='mw-category-generated')
                manga_links = category_div.find_all('a', href=True)

            except:
                category_div = soup.find('div', class_='mw-category-group')
                manga_links = category_div.find_all('a', href=True)

        manga_links = category_div.find_all('a', href=True)

            # 見つかった 'a' タグをループしてタイトルとURLを抽出します
        for link in manga_links:
            title = link.get('title')
            href = 'https://ja.wikipedia.org' + link.get('href')
            manga_list.append([year,title, href])

        # マンガのタイトルとURLのリストを出力します
        for year,title, href in manga_list:
            print(f'{year}年　 {title}, URL: {href}')

        #以下は「次のページがある場合の対策」
        while True:
            next_page_link = soup.find('a', text='次のページ')
            if next_page_link:
                url = 'https://ja.wikipedia.org' + next_page_link.get('href')
                print(f"The URL for '次のページ' is: {url}")

                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'html.parser')
                try:
                    category_div = soup.find('div', class_='mw-category mw-category-columns')
                    manga_links = category_div.find_all('a', href=True)
                except:
                    try:
                        category_div = soup.find('div', class_='mw-category-generated')
                        manga_links = category_div.find_all('a', href=True)

                    except:
                        category_div = soup.find('div', class_='mw-category-group')
                        manga_links = category_div.find_all('a', href=True)
                manga_links = category_div.find_all('a', href=True)

                for link in manga_links:
                    title = link.get('title')
                    href = 'https://ja.wikipedia.org' + link.get('href')
                    manga_list.append([year,title, href])

                # マンガのタイトルとURLのリストを出力します
                for year,title, href in manga_list:
                    print(f'{year}年　 {title}, URL: {href}')

            else:
                #「次のページ」がもうない時
                break

        all_manga_list += manga_list #合体する
    except Exception as e:
        print(e)
        print(f"{year}年：取得失敗")

The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1900%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB


C:\Users\tanigutirei\AppData\Local\Temp\ipykernel_67892\1027902350.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_link = soup.find('a', text='次のページ')


The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1901%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1902%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1903%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1904%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1905%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1906%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1907%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1908%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1909%E5%B9%B4%E3%81%AE%E6%BC%AB%E7%94%BB
The URL for '次のページ' is: https://ja.wikipedia.org/wiki/Category:1

In [3]:
#漫画リストを保存
print(len(all_manga_list ))
import pickle
with open("../manga_title_list.pickle",mode="wb") as f:
    pickle.dump(all_manga_list ,f)

10920


In [4]:
#次にhtmlから情報を取得する
#htmlデータはバイナリデータにしてpickleとして保存します
import urllib.request
import pickle
import requests
from bs4 import BeautifulSoup
import time
import json
with open("../manga_title_list.pickle", mode='br') as fi:
    loaded_manga_list = pickle.load(fi)

# リストを辞書に変換する関数
def list_to_dict(lst):
    result_dict = {}
    for year, title, url in lst:
        if year not in result_dict:
            result_dict[year] = []
        result_dict[year].append([title, url])
    return result_dict

# 関数を使って変換
manga_data_dict_by_year = list_to_dict(loaded_manga_list)

error_log_and_title = [] #いちいちエラー発生して止まってめんどいのでここに保存して後から見よう！

start_year = 1900#一度止めてしまったのでここからスタート

for year,title_and_url in manga_data_dict_by_year.items():
    if start_year > year:
        continue

    file_is_exist = True
    print(year)
    pickle_data = {}
    try:
        with open("../manga_wiki_html_data/" + f"{year}.pickle",mode="br") as f:
            loaded_pickle_data = pickle.load(f)

    except:
        print(f"{year}.pickle" +"が存在しません！")
        file_is_exist = False

    for data in title_and_url:
        title, url = data


        if file_is_exist:
            if title in loaded_pickle_data:
                pickle_data[title] =loaded_pickle_data[title] #基本はロードした情報をそのままコピーするだけ
                continue #time.sleepをスキップする
            else:
                print(f"{year}.pickle は存在しますが、{title} という漫画は保存されていませんでした。そのため辞書に追加します。")
                try:

                    wiki_html = urllib.request.urlopen(url).read().decode('utf-8')#htmlを取得する

                except Exception as e:
                    print(e)
                    error_log_and_title.append([year,title,url,e,"pickleファイルは存在するが漫画が保存されていなかったためrequestを使ってページ取得中に発生"])
                    continue
                pickle_data[title] = wiki_html

        else:
            print(f"{year}.pickleが存在しません。{title}を取得したので後から保存します。")
            try:
                wiki_html = urllib.request.urlopen(url).read().decode('utf-8')#htmlを取得する

            except Exception as e:
                print(e)
                error_log_and_title.append([year,title,url,e,"pickleファイルがなく、辞書に追加するためにrequestを使ってページ取得中に発生"])
                continue
            pickle_data[title] = wiki_html

        time.sleep(1) #一応良心あるんで・・・
    with open("../manga_wiki_html_data/" + f"{year}.pickle",mode="wb") as f:
        pickle.dump(pickle_data ,f)


for data in error_log_and_title:
    print(data)

1930
1931
1932
1933
1934
1935
1936
1937
1938
1938.pickle は存在しますが、ジャングルの女王シーナ という漫画は保存されていませんでした。そのため辞書に追加します。
1939
1940
1941
1942
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1967.pickle は存在しますが、たまねぎたまちゃん という漫画は保存されていませんでした。そのため辞書に追加します。
1968
1969
1970
1970.pickle は存在しますが、美人はいかが? という漫画は保存されていませんでした。そのため辞書に追加します。
1971
1971.pickle は存在しますが、青い季節 (もりたじゅんの漫画) という漫画は保存されていませんでした。そのため辞書に追加します。
1972
1973
1974
1975
1975.pickle は存在しますが、紅い牙 という漫画は保存されていませんでした。そのため辞書に追加します。
1975.pickle は存在しますが、超少女明日香 という漫画は保存されていませんでした。そのため辞書に追加します。
1976
1977
1978
1978.pickle は存在しますが、昆虫皇帝 という漫画は保存されていませんでした。そのため辞書に追加します。
1979
1980
1980.pickle は存在しますが、忍者飛翔 という漫画は保存されていませんでした。そのため辞書に追加します。
1980.pickle は存在しますが、まいっちんぐマチコ先生 という漫画は保存されていませんでした。そのため辞書に追加します。
1981
1982
1983
1983.pickle は存在しますが、あおいちゃんパニック! という漫画は保存されていませんでした。そのため辞書に追加します。
1984
1984.pickle は存在しますが、蒼き守護神 という漫画は保存されていませんでした。そのため辞書に追加します。
1985
1986
1986.pickle は存在しますが、おんな風林火山 という漫画は保存されていませんでした。その

これ以降はwiki-HTMLからdetailsを活用したhtmlに変換するコード